## **NEF Project 5**

**Project Goal and Objectives**: Have the bot every 30 mins from 9am - 9pm EST give an update on the traffic flow and compare those values with MAs.

### **Bot**:
 - 1) Trigger the file request using a trigger: Trigger + Playwirght 
 - 2) Call the notebook (NEF_Project_5_local.ipynb)
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_%_local.ipynb**:
- 1) Will start by impoting the file (Traffic View 3)
- 2) Calculate all the present stats 
- 3) Insert into the databse the values for the given day and hour 
- 4) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 

#### STEP 1 - Importing Traffic View for a given instance 

In [1]:
import os 
import pandas as pd

current_dir = os.getcwd()
files = os.listdir(current_dir)
for file in files:
    if "Project5" in file and file.endswith('.csv'):
        print(file)
        file_name = file
        
tview3 = pd.read_csv(file_name)
tview3.head()



Project5.csv


,Item,Campaign,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP 1,1 - TRAFFIC,"1,596","1,003",62.84%,593,37.16%,"1,076",483,44.89%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$599.96,10.66%,"$4,894.75",86.98%
1,DP 1,2 - INTERNAL,27,11,40.74%,16,59.26%,20,4,20.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$13.02,8.57%,$138.82,91.43%
2,DP 1,3 - PREPAID,249,169,67.87%,80,32.13%,172,92,53.49%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$87.00,11.46%,$653.22,86.04%
3,DP 1 RP,159 - REPROCESS,446,367,82.29%,79,17.71%,411,332,80.78%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$163.12,11.13%,"$1,302.33",88.87%
4,DP 12,1 - TRAFFIC,400,356,89.00%,44,11.00%,369,325,88.08%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$264.38,5.19%,"$4,827.74",94.81%


#### **STEP 2 - Calculate the current statistics**

**Guide**:
- S1 Initials is DP1 from Traffic/Scrub/Prepaid
- S1 RP is DP1RP from Traffic/Scrub/Prepaid
- S1LTV and UPLTV are always static

In [2]:
# We will start off by calculating S1
main_dict = {}
s1_df = tview3[
    (tview3["Item"] == "DP 1") &
    (
        (tview3["Campaign"] == "1 - TRAFFIC") |
        (tview3["Campaign"] == "2 - INTERNAL") |
        (tview3["Campaign"] == "3 - PREPAID")
    )
]
s1_df.head()

# Check if column is string/object type before applying string operations
if s1_df["Successful Charges"].dtype == 'object':
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
else:
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


total_s1 = s1_df["Successful Charges"].sum()
print(f"Total S1 Successful Charges: {total_s1}")

# Get the Gross
S1G = total_s1 * 9.49

# Get the new S1
S1N = S1G * 0.3

main_dict["S1"] = [total_s1, S1G, S1N]

Total S1 Successful Charges: 689


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\1857511017.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


In [3]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


1961.583
2.847


In [4]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


1961.583
2.847


In [5]:
# Now we will calculate S1RP
s1rp_df = tview3[
    (tview3["Item"] == "DP 1 RP") &
    (
        (tview3["Campaign"] == "159 - REPROCESS")

    )
]
s1rp_df.head()
s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 79


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\380405569.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [6]:
# Now we will calculate UP
up_df = tview3[
    (tview3["Item"] == "DP 2") &
    (
        (tview3["Campaign"] == "1 - TRAFFIC") |
        (tview3["Campaign"] == "2 - INTERNAL") |
        (tview3["Campaign"] == "3 - PREPAID")
    )
]
up_df.head()
up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

total_up = up_df["Successful Charges"].sum()
print(f"Total UP Successful Charges: {total_up:,}")

# Get the Gross
UPG = total_up * 38.11

# Get the Net
UPN = UPG * 0.23

main_dict["UP"] = [total_up, UPG, UPN]


Total UP Successful Charges: 226


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\3693429964.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [7]:
# Now we will add the WHC row
whc = total_s1 
whc_g = whc * 3
whc_n = whc_g

# Now we add it to the dic 
main_dict["WHC"] = [whc, whc_g, whc_n]


In [8]:
# NOw we will add UR
ur = total_s1
ur_g = ur * 2
ur_n = ur_g

# Now we add it to the dic 
main_dict["UR"] = [ur, ur_g, ur_n]





In [9]:
# Now ew will add S1 RB Proj
s1rb_proj = (total_s1 + total_s1rp) * 0.2
s1rb_proj_g = s1rb_proj * 115.73
s1rb_proj_n = s1rb_proj_g * 0.25

# Now we add it to the dic 
main_dict["S1RB Proj"] = [s1rb_proj, s1rb_proj_g, s1rb_proj_n]


In [10]:
# Now we will ad UP RB Proj
uprb_proj = total_up * 0.25
uprb_proj_g = uprb_proj * 38.11
uprb_proj_n = uprb_proj_g * 0.5

# Now we add it to the dic 
main_dict["UPRB Proj"] = [uprb_proj, uprb_proj_g, uprb_proj_n]




In [11]:
# Now we will calculate S1RP
s1rp_df = tview3[
    (tview3["Item"] == "DP 1 RP") &
    (
        (tview3["Campaign"] == "159 - REPROCESS")

    )
]
s1rp_df.head()

s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 79


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\471134279.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [12]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


13084.034300000001
18.989890130624094


In [13]:
ecpa_df = tview3[
    (tview3["Item"] == "DP 1") &
    (tview3["Campaign"] == "1 - TRAFFIC")
]
ecpa_df.head()

ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

dp1_traffic = ecpa_df["Successful Charges"].sum()

ecpa = round((dp1_traffic / ((total_s1 + total_s1rp))) * -30, 2)
print(ecpa)


-23.16


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\1560642248.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [14]:
S1LTV = 3.50
UPLTV = 1.5




gross_total = 0
for key, values in main_dict.items():
    gross_total += values[1]
print(gross_total)

middle = main_dict["WHC"][1] + main_dict["UR"][1]

RES = (((gross_total - middle)*0.1)/total_s1)+1
print(RES)


NRC = NRCPA + ecpa + S1LTV + UPLTV + RES
print(NRC)







upsell_take = round((total_up / (total_s1))*100, 2)
print(upsell_take)






39991.263000000006
6.304247169811322
7.134137300435415
32.8


In [15]:
# And in S1ltv
s1ltv = total_s1
s1ltv_g = s1ltv * 4
s1ltv_n = s1ltv_g 

# Add to the dict 
main_dict["S1LTV"] = [s1ltv, s1ltv_g, s1ltv_n]


In [16]:
# And upltv
upltv = total_s1
upltv_g = upltv * 2
upltv_n = upltv_g 

# Add to the dict 
main_dict["UPLTV"] = [upltv, upltv_g, upltv_n]




In [17]:
# Add iin the res 
res = total_s1
res_g = (((((main_dict["S1"][1] + main_dict["S1RP"][1] + main_dict["UP"][1] + main_dict["S1RB Proj"][1] + main_dict["UPRB Proj"][1]) * 0.1) / total_s1) + 3) * total_s1)
res_n = res_g 

# Add to the dict 
main_dict["RES"] = [res, res_g, res_n]




NRC =  ecpa 
print(NRC)


-23.16


In [18]:
# Now we need to add in eCPA
eCPA =  dp1_traffic 
eCPA_g = eCPA * -30
eCPA_n = eCPA_g 

# Add to the dict 
main_dict["eCPA"] = [eCPA, eCPA_g, eCPA_n]  


sum_for_nrc_new = 0
for key, values in main_dict.items():
    if key != "eCPA":
        sum_for_nrc_new += values[2]

nrc_new = sum_for_nrc_new / (total_s1 + total_s1rp)
nrc_new



29.86934973958333

In [19]:
main_dict

{'S1': [689, 6538.610000000001, 1961.583],
 'S1RP': [79, 1465.45, 175.85399999999998],
 'UP': [226, 8612.86, 1980.9578000000001],
 'WHC': [689, 2067, 2067],
 'UR': [689, 1378, 1378],
 'S1RB Proj': [153.60000000000002, 17776.128000000004, 4444.032000000001],
 'UPRB Proj': [56.5, 2153.215, 1076.6075],
 'S1LTV': [689, 2756, 2756],
 'UPLTV': [689, 1378, 1378],
 'RES': [689, 5721.626300000001, 5721.626300000001],
 'eCPA': [593, -17790, -17790]}

In [20]:
# TV analysis message 
message = "📺 TV message:\n"

# Add teh number of customer, which is total_s1 + total_s1rp
message += f"• Number of customers: {total_s1 + total_s1rp:,}\n"

# Add the upsell take, 
message += f"• Upsell take: {upsell_take:,}%\n"

# Add the ecpa
message += f"• eCPA: {ecpa:,}\n"

# Add the NRC
nrc = nrc_new + ecpa
message += f"• NR/C: {round(nrc, 2):,}\n"


print(message)
list_current = []
list_current.append(total_s1 + total_s1rp)
list_current.append(upsell_take)
list_current.append(ecpa)
list_current.append(nrc)



present_value_customers = (total_s1 + total_s1rp)
present_values_upsell_take = upsell_take
present_value_ecpa = ecpa
present_value_nrc = nrc 

📺 TV message:
• Number of customers: 768
• Upsell take: 32.8%
• eCPA: -23.16
• NR/C: 6.71



### Import the Dataframe 

In [21]:
database_df = pd.read_excel("tv_database.xlsx")

# Create a column where wonly have the date and no time 
database_df["Date"] = database_df["Date-Time"].dt.date

database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP 1,DP 1 RP,DP 2
0,2025-06-04 21:30:00,1053,43.11,-12.11,18.391400,2025-06-04,21:30:00,2428,798,1170
1,2025-06-04 22:00:00,1064,44.22,-12.04,18.585741,2025-06-04,22:00:00,2448,819,1226
2,2025-06-04 22:30:00,1072,44.64,-12.03,18.624457,2025-06-04,22:30:00,2462,833,1279
3,2025-06-04 23:00:00,1080,45.33,-12.06,18.686259,2025-06-04,23:00:00,2488,848,1328
4,2025-06-04 23:30:00,1092,45.43,-12.03,18.709051,2025-06-04,23:30:00,2509,865,1379
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-17 03:30:00,1446,43.80,-22.76,8.460419,2025-06-17,03:30:00,3649,962,2014
438,2025-06-17 03:30:00,1446,43.80,-22.76,8.460419,2025-06-17,03:30:00,3649,962,2014
439,2025-06-17 03:00:00,1446,43.80,-22.76,8.460419,2025-06-17,03:00:00,3649,962,2014
440,2025-06-17 03:30:00,1564,43.69,-22.92,8.387123,2025-06-17,03:30:00,3971,1017,2151


In [22]:
# Get the average of the customers, upsell, ecpa, nrc
real_average = database_df["customers"].mean()
real_average_upsell = database_df["upsell"].mean()
real_average_ecpa = database_df["excpa"].mean()
real_average_nrc = database_df["nrc"].mean()



print(real_average)
print(real_average_upsell)
print(real_average_ecpa)
print(real_average_nrc)





577.4864253393665
inf
-17.214125874125877
13.263767723576267


In [23]:
# Get the rounded hour / half hour interval 
from datetime import datetime, timedelta

def get_current_rounded_time(x):
    """
    Get current time rounded to nearest half hour.
    Returns datetime object rounded to nearest half hour.
    """
    now = x
    
    # Calculate minutes to add/subtract to round to nearest half hour
    minutes = now.minute
    if minutes < 15:
        # Round down to previous half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0)
    elif minutes < 45:
        # Round to current half hour
        rounded_time = now.replace(minute=30, second=0, microsecond=0)
    else:
        # Round up to next half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    
    return rounded_time
timey = datetime.now()
datetimee = get_current_rounded_time(timey)
print(datetimee)
database_df.shape

2025-06-17 18:00:00


(442, 10)

In [24]:
from datetime import datetime
from zoneinfo import ZoneInfo
timeest = timey = datetime.now(ZoneInfo("America/New_York"))
timei = get_current_rounded_time(timeest)
timei.hour, timei.minute
timei = timei.strftime("%H:%M")
timei


'13:00'

### In this section we integartes the Attempts for DP1, DP1 RP, DP 2 etc

In [25]:
tview3

,Item,Campaign,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP 1,1 - TRAFFIC,"1,596","1,003",62.84%,593,37.16%,"1,076",483,44.89%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$599.96,10.66%,"$4,894.75",86.98%
1,DP 1,2 - INTERNAL,27,11,40.74%,16,59.26%,20,4,20.00%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$13.02,8.57%,$138.82,91.43%
2,DP 1,3 - PREPAID,249,169,67.87%,80,32.13%,172,92,53.49%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$87.00,11.46%,$653.22,86.04%
3,DP 1 RP,159 - REPROCESS,446,367,82.29%,79,17.71%,411,332,80.78%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$163.12,11.13%,"$1,302.33",88.87%
4,DP 12,1 - TRAFFIC,400,356,89.00%,44,11.00%,369,325,88.08%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$264.38,5.19%,"$4,827.74",94.81%
5,DP 12,2 - INTERNAL,65,53,81.54%,12,18.46%,62,50,80.65%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$72.29,5.21%,"$1,200.74",86.46%
6,DP 12,3 - PREPAID,90,85,94.44%,5,5.56%,86,81,94.19%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$33.14,5.73%,$545.51,94.27%
7,DP 12 RP,159 - REPROCESS,77,65,84.42%,12,15.58%,69,57,82.61%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$60.71,4.37%,"$1,212.32",87.30%
8,DP 13,1 - TRAFFIC,35,31,88.57%,4,11.43%,34,30,88.24%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$23.92,6.15%,$364.80,93.85%
9,DP 13,2 - INTERNAL,29,23,79.31%,6,20.69%,27,21,77.78%,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$12.52,2.15%,$570.56,97.85%


In [26]:
### In this section we integrate the Attempts for DP1, DP1 RP, DP2 etc

# DP 1
dddp1_df = tview3[tview3["Item"] == 'DP 1']
dddp1_df = dddp1_df[dddp1_df["Campaign"] != "159 - REPROCESS"]
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(str)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(float)
averagedp1 = dddp1_df["Attempted Charges"].sum()
print(averagedp1)

# DP 1 RP
dddp1rp_df = tview3[tview3["Item"] == 'DP 1 RP']
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(str)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(float)
averagedp1rp = dddp1rp_df["Attempted Charges"].sum()
print(averagedp1rp)

# DP 2
dddp2_df = tview3[tview3["Item"] == 'DP 2']
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(str)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(float)
averagedp2 = dddp2_df["Attempted Charges"].sum()
print(averagedp2)

# Append these to the list_current
list_current.append(averagedp1)
list_current.append(averagedp1rp)
list_current.append(averagedp2)


1872.0
446.0
670.0


C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\775290.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(str)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\775290.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].str.replace(',', '', regex=False)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_36408\775290.py:16: SettingWithCopyWarning: 
A value is trying to be s

In [27]:
database_df.columns

Index(['Date-Time', 'customers', 'upsell', 'excpa', 'nrc', 'Date', 'Time',
       'DP 1', 'DP 1 RP', 'DP 2'],
      dtype='object')

In [28]:
# Create a new row as a dictionary
new_row = {
    'Date-Time': datetimee,
    'customers': present_value_customers,
    'upsell': present_values_upsell_take,
    'excpa': present_value_ecpa,
    'nrc': present_value_nrc,
    'Date': datetimee.now().date(),
    "Time": datetimee.now().time(),
    "DP 1": averagedp1,
    "DP 1 RP": averagedp1rp,
    "DP 2": averagedp2

}

# Add the row using loc
database_df.loc[len(database_df)] = new_row

# Now we drop the oldest one
database_df.drop(index=0, inplace=True)
database_df = database_df.reset_index(drop=True)
database_df["Time"] = database_df["Date-Time"].dt.time


database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP 1,DP 1 RP,DP 2
0,2025-06-04 22:00:00,1064,44.22,-12.04,18.585741,2025-06-04,22:00:00,2448.0,819.0,1226.0
1,2025-06-04 22:30:00,1072,44.64,-12.03,18.624457,2025-06-04,22:30:00,2462.0,833.0,1279.0
2,2025-06-04 23:00:00,1080,45.33,-12.06,18.686259,2025-06-04,23:00:00,2488.0,848.0,1328.0
3,2025-06-04 23:30:00,1092,45.43,-12.03,18.709051,2025-06-04,23:30:00,2509.0,865.0,1379.0
4,2025-06-05 00:00:00,1104,45.81,-12.09,18.704132,2025-06-05,00:00:00,2532.0,881.0,1433.0
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-17 03:30:00,1446,43.80,-22.76,8.460419,2025-06-17,03:30:00,3649.0,962.0,2014.0
438,2025-06-17 03:00:00,1446,43.80,-22.76,8.460419,2025-06-17,03:00:00,3649.0,962.0,2014.0
439,2025-06-17 03:30:00,1564,43.69,-22.92,8.387123,2025-06-17,03:30:00,3971.0,1017.0,2151.0
440,2025-06-17 18:00:00,768,32.80,-23.16,6.709350,2025-06-17,18:00:00,1872.0,446.0,670.0


##### Get the average values for each half an hour interval

In [29]:
from datetime import time
unique_times = database_df["Time"].unique().tolist()
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

# Create the percenatge change function 
def percentage_change(new, old):
    if new != old:
        p = ((new - old)/old)*100
        return p
    if new == 0 or old == 0:
        return "Invalid"
    else:
        return 1

def alert_maker(list_of_mass, list_of_currentt):
    name_of_componets = ["Customer Sales", "Upsell", "Ecpa", "Nrc", "DP 1", "DP 1 RP", "DP 2"]
    alerts = {}
    alerts_cons = {}
    if not len(list_current) == len(list_mas):
        return None, None
    
    for i in range(len(list_current)):
        new = list_of_currentt[i]
        old = list_of_mass[i]
        diff = percentage_change(new, old)
        diff = round(diff, 2)

        if abs(diff) > 20:
            alerts[name_of_componets[i]] = [new, diff]
        if abs(diff) == 1:
            alerts_cons[name_of_componets[i]] = [new, diff]

    # Return empty dicts instead of None if no alerts
    if not alerts:
        alerts = {}
    if not alerts_cons:
        alerts_cons = {}
        
    print(alerts_cons)
    exclude_keys = {"DP 1", "DP 1 RP", "DP 2"}
    alerts = {k: v for k, v in alerts.items() if k not in exclude_keys}
    return alerts, alerts_cons




print(datetimee.hour, datetimee.minute)
df = database_df[database_df["Time"] == time(datetimee.hour, datetimee.minute)]


# Create a list with all the moving averages
list_mas = []

moving_average_customers = df["customers"].mean()
list_mas.append(moving_average_customers)
moving_average_upsell = df["upsell"].mean()
list_mas.append(moving_average_upsell)
moving_average_ecpa = df["excpa"].mean()
list_mas.append(moving_average_ecpa)
moving_average_ncr = df["nrc"].mean()
list_mas.append(moving_average_ncr)

# Get the previous attemot values for the varipus DPs
df_prev = database_df.iloc[[-2]]
moving_averagedp1 = df_prev["DP 1"].values[0]
list_mas.append(moving_averagedp1)

moving_averagedp1rp = df_prev["DP 1 RP"].values[0]
list_mas.append(moving_averagedp1rp)

moving_averagedp2 = df_prev["DP 2"].values[0]
list_mas.append(moving_averagedp2)

print(f"The values for dp1, dp1rp and dp2 are {moving_averagedp1}, {moving_averagedp1rp}, {moving_averagedp2}")
print()


alert_component = alert_maker(list_mas, list_current)
print(f"The current values for the moving averages are: {list_mas}")
print(f"The current values of this analysis are: {list_current}")
print(alert_component[0])
print(alert_component[1])

18233.694900000002
18 0
The values for dp1, dp1rp and dp2 are 1872.0, 446.0, 670.0

{'DP 1': [1872.0, 1], 'DP 1 RP': [446.0, 1], 'DP 2': [670.0, 1]}
The current values for the moving averages are: [597.0, 34.80571428571429, -18.571428571428573, 10.819937134831152, 1872.0, 446.0, 670.0]
The current values of this analysis are: [768, 32.8, -23.16, 6.709349739583331, 1872.0, 446.0, 670.0]
{'Customer Sales': [768, 28.64], 'Ecpa': [-23.16, 24.71], 'Nrc': [6.709349739583331, -37.99]}
{'DP 1': [1872.0, 1], 'DP 1 RP': [446.0, 1], 'DP 2': [670.0, 1]}


In [30]:
print(alert_component[0])

{'Customer Sales': [768, 28.64], 'Ecpa': [-23.16, 24.71], 'Nrc': [6.709349739583331, -37.99]}


In [31]:
message += "\n"
output_message = message
if alert_component[0] or alert_component[1]:
    if alert_component[0]:
        message += "⚠️We have detected alerts: \n"
        for key, value in alert_component[0].items():
            message += f'• {key} has a deviaition of {value[1]}% from its respective moving average at {timei}. \n'
        output_message = message
    if alert_component[1]:
        for key, value in alert_component[1].items():
            output_message += f'• {key} attempts are equal to those half an hour ago. \n'

print(output_message)


📺 TV message:
• Number of customers: 768
• Upsell take: 32.8%
• eCPA: -23.16
• NR/C: 6.71

⚠️We have detected alerts: 
• Customer Sales has a deviaition of 28.64% from its respective moving average at 13:00. 
• Ecpa has a deviaition of 24.71% from its respective moving average at 13:00. 
• Nrc has a deviaition of -37.99% from its respective moving average at 13:00. 
• DP 1 attempts are equal to those half an hour ago. 
• DP 1 RP attempts are equal to those half an hour ago. 
• DP 2 attempts are equal to those half an hour ago. 



In [32]:
# Delete the file tv_database.xlsx from the directory 
file_path = "tv_database.xlsx"
os.remove(file_path)

# Save the current database_df as tv_database.xlsx
database_df.to_excel("tv_database.xlsx", index=False)